# Abalone Project - Model Classification
Maria Eugênia Fonseca\
2021/10/05

In [1]:
import pandas as pd
import mlflow

from sklearn.metrics import accuracy_score, f1_score
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
X_train = pd.read_csv("../data/processed/abalone_xtrain_class.csv")
y_train = pd.read_csv("../data/processed/abalone_ytrain_class.csv")

X_test = pd.read_csv("../data/processed/abalone_xtest_class.csv")
y_test = pd.read_csv("../data/processed/abalone_ytest_class.csv")

In [3]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9
0,0.540541,0.521008,0.093750,0.176023,0.155346,0.115866,0.153961,1.0,0.0,0.0
1,0.601351,0.621849,0.125000,0.287232,0.258238,0.264648,0.227703,0.0,1.0,0.0
2,0.736486,0.714286,0.151786,0.399681,0.354741,0.348914,0.303438,1.0,0.0,0.0
3,0.378378,0.394958,0.071429,0.115282,0.089442,0.112574,0.088191,0.0,0.0,1.0
4,0.452703,0.436975,0.075893,0.098636,0.075992,0.044766,0.108122,1.0,0.0,0.0


In [4]:
y_train.head()

,age
0,middle age
1,middle age
2,middle age
3,middle age
4,middle age


#### Start mlflow experiment:

In [5]:
EXPERIMENT_NAME = "abalone_classification"

try:
    mlflow.create_experiment(EXPERIMENT_NAME)
    print("CREATING")
except:
    print("ALREADY EXISTS")

mlflow.set_experiment(EXPERIMENT_NAME)

ALREADY EXISTS


#### Function to evaluate classification and log metrics in mlflow:

In [6]:
def evaluate_classification_and_log(classification_model, X_train, y_train, X_test, y_test, params):    
    # metrics - train
    y_train_pred = classification_model.predict(X_train)
    accuracy_train = accuracy_score(y_train, y_train_pred)
    f1_train = f1_score(y_train, y_train_pred, average='weighted')

    # metrics - test
    y_test_pred = classification_model.predict(X_test)
    accuracy_test = accuracy_score(y_test, y_test_pred)
    f1_test = f1_score(y_test, y_test_pred, average='weighted')
    
    # log metrics    
    mlflow.start_run()

    mlflow.log_metric("accuracy_train", accuracy_train)
    mlflow.log_metric("f1_train", f1_train)
    mlflow.log_metric("accuracy_test", accuracy_test)
    mlflow.log_metric("f1_test", f1_test)

    mlflow.sklearn.log_model(classification_model, "model")
    mlflow.log_param("model_name", type(classification_model).__name__)
    mlflow.log_params(params)
    
    mlflow.end_run()

#### First model - CategoricalNB

In [7]:
params_pipe = {
    "minmax_scaller": "true"}

In [8]:
nb = CategoricalNB().fit(X_train, y_train)

evaluate_classification_and_log(nb, X_train, y_train, X_test, y_test, params_pipe)

C:\Users\fonse\anaconda3\envs\abalone\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Second model - KNeighborsClassifier

In [9]:
knn = KNeighborsClassifier().fit(X_train, y_train)

evaluate_classification_and_log(knn, X_train, y_train, X_test, y_test, params_pipe)

C:\Users\fonse\anaconda3\envs\abalone\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\fonse\anaconda3\envs\abalone\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\fonse\anaconda3\envs\abalone\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [10]:
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

#### Third model - KNeighborsClassifier with hyperparameter tuning

In [11]:
params_pipe = {
    "minmax_scaller": "true",
    "hyperparameter_tuning": "true"}

In [12]:
n_neighbors = [4, 5, 6, 7, 8]
weights = ['uniform', 'distance']

# Create the grid
search_grid = {'n_neighbors': n_neighbors,
               'weights': weights}

In [13]:
knn = KNeighborsClassifier()

# Grid search of parameters, using 3 fold cross validation, 
knn_tuning = GridSearchCV(estimator=knn, param_grid=search_grid, cv=3, n_jobs=-1)

# Fit the random search model
knn_tuning.fit(X_train, y_train)

C:\Users\fonse\anaconda3\envs\abalone\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [4, 5, 6, 7, 8],
                         'weights': ['uniform', 'distance']})

In [14]:
knn_tuning.best_params_

{'n_neighbors': 7, 'weights': 'uniform'}

In [15]:
knn_tuned = knn_tuning.best_estimator_

evaluate_classification_and_log(knn_tuned, X_train, y_train, X_test, y_test, params_pipe)

C:\Users\fonse\anaconda3\envs\abalone\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\fonse\anaconda3\envs\abalone\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


#### Fourth model - RandomForestClassifier no tuning

In [16]:
params_pipe = {
    "minmax_scaller": "true",
    "hyperparameter_tuning": "false"}

In [17]:
rf = RandomForestClassifier(random_state=123).fit(X_train, y_train)

evaluate_classification_and_log(rf, X_train, y_train, X_test, y_test, params_pipe)

C:\Users\fonse\AppData\Local\Temp/ipykernel_15200/1843286035.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf = RandomForestClassifier(random_state=123).fit(X_train, y_train)


In [18]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 123,
 'verbose': 0,
 'warm_start': False}

#### Fith model - RandomForestClassifier with hyperparameter tuning

In [19]:
params_pipe = {
    "minmax_scaller": "true",
    "hyperparameter_tuning": "true"}

In [20]:
n_estimators = [115, 120, 125]
max_depth = [6, 7, 8]
min_samples_leaf = [6, 7, 8]
ccp_alpha = [0.0009, 0.001, 0.003]
max_samples = [0.68, 0.7, 0.72]

# Create the grid
search_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
               'ccp_alpha': ccp_alpha,
               'max_samples': max_samples
              }

In [21]:
rf = RandomForestClassifier(random_state=123)

# Grid search of parameters, using 3 fold cross validation, 
rf_tuning = GridSearchCV(estimator=rf, param_grid=search_grid, cv=3, n_jobs=-1)

# Fit the random search model
rf_tuning.fit(X_train, y_train)

C:\Users\fonse\anaconda3\envs\abalone\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=123),
             n_jobs=-1,
             param_grid={'ccp_alpha': [0.0009, 0.001, 0.003],
                         'max_depth': [6, 7, 8],
                         'max_samples': [0.68, 0.7, 0.72],
                         'min_samples_leaf': [6, 7, 8],
                         'n_estimators': [115, 120, 125]})

In [22]:
rf_tuning.best_params_

{'ccp_alpha': 0.0009,
 'max_depth': 8,
 'max_samples': 0.72,
 'min_samples_leaf': 7,
 'n_estimators': 115}

In [23]:
rf_tuned = rf_tuning.best_estimator_

evaluate_classification_and_log(rf_tuned, X_train, y_train, X_test, y_test, params_pipe)

#### Sixth model - GradientBoostingClassifier with hyperparameter tuning

In [24]:
params_pipe = {
    "minmax_scaller": "true",
    "hyperparameter_tuning": "false"}

In [25]:
gb = GradientBoostingClassifier(random_state=123).fit(X_train, y_train)

evaluate_classification_and_log(gb, X_train, y_train, X_test, y_test, params_pipe)

C:\Users\fonse\anaconda3\envs\abalone\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Seventh model - GradientBoostingClassifier with hyperparameter tuning

In [26]:
params_pipe = {
    "minmax_scaller": "true",
    "hyperparameter_tuning": "true"}

In [ ]:
learning_rate = [0.05, 0.1]
n_estimators = [100, 120, 140]
subsample = [0.6, 0.7, 0.8]
min_samples_leaf = [6, 7, 8]
max_depth = [6, 7, 8]
ccp_alpha = [0.0001, 0.001, 0.01]

# Create the grid
search_grid = {'learning_rate': learning_rate,
               'n_estimators': n_estimators,
               'subsample': subsample,
               'min_samples_leaf': min_samples_leaf,
               'max_depth': max_depth,
               'ccp_alpha': ccp_alpha
              }

In [ ]:
gb = GradientBoostingClassifier(random_state=123)

# Grid search of parameters, using 3 fold cross validation, 
gb_tuning = GridSearchCV(estimator=gb, param_grid=search_grid, cv=3, n_jobs=-1)

# Fit the random search model
gb_tuning.fit(X_train, y_train)

In [ ]:
gb_tuning.

In [ ]:
gb_tuned = gb_tuning.best_estimator_

evaluate_classification_and_log(gb_tuned, X_train, y_train, X_test, y_test, params_pipe)